<a href="https://colab.research.google.com/github/chavalvit-k/Tweet-Location-Classifier-with-Naive-Bayes/blob/main/Tweet_Location_Classifier_with_Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [194]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from nltk.stem import PorterStemmer

pd.set_option("display.max_columns", None)

# Import Dataset

In [9]:
london_tweets = pd.read_json("london.json", lines = True)
new_york_tweets = pd.read_json("new_york.json", lines = True)
paris_tweets = pd.read_json("paris.json", lines = True)

In [10]:
london_tweets.head()

,created_at,id,id_str,text,display_text_range,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,geo,coordinates,place,contributors,is_quote_status,extended_tweet,quote_count,reply_count,retweet_count,favorite_count,entities,favorited,retweeted,filter_level,lang,timestamp_ms,possibly_sensitive,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,extended_entities
0,2018-07-26 13:39:30+00:00,1022476504855400449,1022476504855400448,@bbclaurak i agree Laura but the Party you see...,"[11, 140]","<a href=""http://twitter.com/download/iphone"" r...",True,1.022447e+18,1.022447e+18,6.118357e+07,6.118357e+07,bbclaurak,"{'id': 340170806, 'id_str': '340170806', 'name...",None,None,"{'id': '58f909abfd95e133', 'url': 'https://api...",NaN,False,{'full_text': '@bbclaurak i agree Laura but th...,0,0,0,0,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",False,False,low,en,2018-07-26 13:39:30.109,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-07-26 13:39:30+00:00,1022476506075942912,1022476506075942912,@masturbacaolove Why?,"[17, 21]","<a href=""http://twitter.com/download/iphone"" r...",False,1.021997e+18,1.021997e+18,9.003777e+17,9.003777e+17,masturbacaolove,"{'id': 200812090, 'id_str': '200812090', 'name...",None,None,"{'id': '457b4814b4240d87', 'url': 'https://api...",NaN,False,NaN,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,und,2018-07-26 13:39:30.400,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-07-26 13:39:31+00:00,1022476510089949190,1022476510089949184,@JackRobinson80 @pgroresearch Yeah not great b...,"[30, 65]","<a href=""http://twitter.com/download/iphone"" r...",False,1.022444e+18,1.022444e+18,7.355633e+08,7.355633e+08,JackRobinson80,"{'id': 771664493898182656, 'id_str': '77166449...",None,None,"{'id': '3bc1b6cfd27ef7f6', 'url': 'https://api...",NaN,False,NaN,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,2018-07-26 13:39:31.357,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-07-26 13:39:33+00:00,1022476519845883905,1022476519845883904,Penalty shit out Arsenal,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,None,"{'id': 968613220893290496, 'id_str': '96861322...",None,None,"{'id': '544762ebf7fda780', 'url': 'https://api...",NaN,False,NaN,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,2018-07-26 13:39:33.683,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-07-26 13:39:36+00:00,1022476532684648448,1022476532684648448,Obviously need some pen practice 🙈,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,None,"{'id': 787646625942478848, 'id_str': '78764662...",None,None,"{'id': '09ba692cbddfc92c', 'url': 'https://api...",NaN,False,NaN,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,2018-07-26 13:39:36.744,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
new_york_tweets.head()

,created_at,id,id_str,text,display_text_range,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,geo,coordinates,place,contributors,is_quote_status,quote_count,reply_count,retweet_count,favorite_count,entities,favorited,retweeted,filter_level,lang,timestamp_ms,extended_tweet,possibly_sensitive,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,extended_entities,withheld_in_countries
0,2018-07-26 13:32:33+00:00,1022474755625164800,1022474755625164800,@DelgadoforNY19 Calendar marked.,"[16, 32]","<a href=""http://twitter.com/download/android"" ...",False,1.022208e+18,1.022208e+18,8.290618e+17,8.290618e+17,DelgadoforNY19,"{'id': 316616881, 'id_str': '316616881', 'name...",None,None,"{'id': '01a9a39529b27f36', 'url': 'https://api...",NaN,False,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,2018-07-26 13:32:33.060,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-07-26 13:32:34+00:00,1022474762491183104,1022474762491183104,petition to ban more than one spritz of cologne,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,None,"{'id': 23692064, 'id_str': '23692064', 'name':...",None,None,"{'id': '01a9a39529b27f36', 'url': 'https://api...",NaN,False,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,2018-07-26 13:32:34.697,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-07-26 13:32:35+00:00,1022474765750226945,1022474765750226944,People really be making up beef with you in th...,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,None,"{'id': 219516383, 'id_str': '219516383', 'name...",None,None,"{'id': '8479859139338fb7', 'url': 'https://api...",NaN,False,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,2018-07-26 13:32:35.474,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-07-26 13:32:36+00:00,1022474768736546816,1022474768736546816,30 years old.. wow what a journey... I moved t...,NaN,"<a href=""http://instagram.com"" rel=""nofollow"">...",True,NaN,NaN,NaN,NaN,None,"{'id': 294201642, 'id_str': '294201642', 'name...","{'type': 'Point', 'coordinates': [40.76074541,...","{'type': 'Point', 'coordinates': [-73.99905224...","{'id': '01a9a39529b27f36', 'url': 'https://api...",NaN,False,0,0,0,0,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",False,False,low,en,2018-07-26 13:32:36.186,{'full_text': '30 years old.. wow what a journ...,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-07-26 13:32:36+00:00,1022474769260838913,1022474769260838912,At first glance it looked like asparagus with ...,NaN,"<a href=""http://twitter.com/download/iphone"" r...",True,NaN,NaN,NaN,NaN,None,"{'id': 455408518, 'id_str': '455408518', 'name...",None,None,"{'id': '01a9a39529b27f36', 'url': 'https://api...",NaN,False,0,0,0,0,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",False,False,low,en,2018-07-26 13:32:36.311,{'full_text': 'At first glance it looked like ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
paris_tweets.head()

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,geo,coordinates,place,contributors,is_quote_status,quote_count,reply_count,retweet_count,favorite_count,entities,favorited,retweeted,filter_level,lang,timestamp_ms,display_text_range,extended_entities,possibly_sensitive,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,extended_tweet
0,2018-07-27 17:40:45+00:00,1022899608396156928,1022899608396156928,Bulletin météo parisien : des grêlons énormes ...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,None,"{'id': 898983688960167936, 'id_str': '89898368...",None,None,"{'id': '09f6a7707f18e0b1', 'url': 'https://api...",NaN,False,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,fr,2018-07-27 17:40:45.854,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-07-27 17:40:47+00:00,1022899613550956544,1022899613550956544,Prêt pour le match #USORCL https://t.co/V5jw0S...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,None,"{'id': 734447481937920002, 'id_str': '73444748...",None,None,"{'id': '70ce4ca586238ac4', 'url': 'https://api...",NaN,False,0,0,0,0,"{'hashtags': [{'text': 'USORCL', 'indices': [1...",False,False,low,fr,2018-07-27 17:40:47.083,"[0, 26]","{'media': [{'id': 1022899599336525825, 'id_str...",0.0,NaN,NaN,NaN,NaN,NaN
2,2018-07-27 17:40:50+00:00,1022899626041651200,1022899626041651200,MAIS QOIDBDNND'SLS'SLSLLSLS''D DBDODNDNODJDBKD...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,None,"{'id': 2161235005, 'id_str': '2161235005', 'na...",None,None,"{'id': '09f6a7707f18e0b1', 'url': 'https://api...",NaN,False,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,in,2018-07-27 17:40:50.061,"[0, 111]","{'media': [{'id': 1022899571884744706, 'id_str...",0.0,NaN,NaN,NaN,NaN,NaN
3,2018-07-27 17:40:57+00:00,1022899655347249152,1022899655347249152,@ToursFC Où peut on le championnat de National...,"<a href=""http://twitter.com/download/android"" ...",False,1.022888e+18,1.022888e+18,978599220.0,978599220.0,ToursFC,"{'id': 814872139, 'id_str': '814872139', 'name...",None,None,"{'id': '23cf420f1650642a', 'url': 'https://api...",NaN,False,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,fr,2018-07-27 17:40:57.048,"[9, 50]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-07-27 17:40:57+00:00,1022899656685223936,1022899656685223936,Les tismey ils sont bas qu’a tromper leur go e...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,None,"{'id': 947517995047424006, 'id_str': '94751799...",None,None,"{'id': '3f712f3df88cbeaf', 'url': 'https://api...",NaN,False,0,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,fr,2018-07-27 17:40:57.367,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Explore Dataset

In [13]:
london_tweets.shape

(5341, 35)

In [14]:
new_york_tweets.shape

(4723, 36)

In [15]:
paris_tweets.shape

(2510, 35)

In [16]:
london_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5341 entries, 0 to 5340
Data columns (total 35 columns):
 #   Column                     Non-Null Count  Dtype              
---  ------                     --------------  -----              
 0   created_at                 5341 non-null   datetime64[ns, UTC]
 1   id                         5341 non-null   int64              
 2   id_str                     5341 non-null   int64              
 3   text                       5341 non-null   object             
 4   display_text_range         3535 non-null   object             
 5   source                     5341 non-null   object             
 6   truncated                  5341 non-null   bool               
 7   in_reply_to_status_id      2230 non-null   float64            
 8   in_reply_to_status_id_str  2230 non-null   float64            
 9   in_reply_to_user_id        2444 non-null   float64            
 10  in_reply_to_user_id_str    2444 non-null   float64            
 11  in_r

In [17]:
new_york_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4723 entries, 0 to 4722
Data columns (total 36 columns):
 #   Column                     Non-Null Count  Dtype              
---  ------                     --------------  -----              
 0   created_at                 4723 non-null   datetime64[ns, UTC]
 1   id                         4723 non-null   int64              
 2   id_str                     4723 non-null   int64              
 3   text                       4723 non-null   object             
 4   display_text_range         2811 non-null   object             
 5   source                     4723 non-null   object             
 6   truncated                  4723 non-null   bool               
 7   in_reply_to_status_id      1668 non-null   float64            
 8   in_reply_to_status_id_str  1668 non-null   float64            
 9   in_reply_to_user_id        1829 non-null   float64            
 10  in_reply_to_user_id_str    1829 non-null   float64            
 11  in_r

In [18]:
paris_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2510 entries, 0 to 2509
Data columns (total 35 columns):
 #   Column                     Non-Null Count  Dtype              
---  ------                     --------------  -----              
 0   created_at                 2510 non-null   datetime64[ns, UTC]
 1   id                         2510 non-null   int64              
 2   id_str                     2510 non-null   int64              
 3   text                       2510 non-null   object             
 4   source                     2510 non-null   object             
 5   truncated                  2510 non-null   bool               
 6   in_reply_to_status_id      1040 non-null   float64            
 7   in_reply_to_status_id_str  1040 non-null   float64            
 8   in_reply_to_user_id        1101 non-null   float64            
 9   in_reply_to_user_id_str    1101 non-null   float64            
 10  in_reply_to_screen_name    1101 non-null   object             
 11  user

# Preprocess

## Combine all text together, and assign the labels

In [182]:
london_text = london_tweets["text"].tolist()
new_york_text = new_york_tweets["text"].tolist()
paris_text = paris_tweets["text"].tolist()

X = london_text + new_york_text + paris_text

y = [0] * len(london_text) + [1] * len(new_york_text) + [2] * len(paris_text)

In [183]:
len(X)

12574

In [184]:
X[0 : len(X) - 1 : 1000]

['@bbclaurak i agree Laura but the Party you seem to support so strongly is slowly doing the same thing . . . and usi… https://t.co/tsRsVBozIR',
 'もしかしてあれか、躁鬱ちゃんと私合わせてㄘんㄘんってことなのか🤔',
 "@Adriana5619 @Winston_Dunhill En un medio de comunicación? Sí? Como también escuchar 'me chupa la poronga'? Sí? Qué loco..",
 '@jordandias Bank, City of London!\n\n(although we are likely to head to India for part of our honeymoon later in the year).',
 '@KhSaad_Rafique نہیں خواجہ صاحب ، آپکی پارٹی کو کراچی والوں کی بد دعائیں لگی ھیں ورنہ پیسہ کی کمی آپکے پاس بھی نہیں… https://t.co/Db3TBzAsfF',
 'You said grime and posted this with your chest???? Americans are mad https://t.co/tViAAGJWdR',
 'Such disrespect for Queens.  The Kingslayer branch does not even warrant a mention.  https://t.co/Q68DVING6D',
 'ugh, I love my momma 🌻',
 'The Pesident is discriminating against Muslim Turkey by lashing out at their religious intolerance! Grounds for imp… https://t.co/S0Af9FmsXw',
 '@Arn68353558Arn Thanks for followi

In [185]:
y[0 : len(y) - 1 : 1000]

[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2]

## Stem Text

In [290]:
porter_stemmer = PorterStemmer()

def stem_text(tweets):
  stemmed_text_list = []

  for tweet in tweets:
    words = tweet.split()
    #stemmed_words = [porter_stemmer.stem(word) for word in words if not word.startswith(("@", "http"))]
    stemmed_words = [porter_stemmer.stem(word) for word in words]
    stemmed_text = " ".join(stemmed_words)
    stemmed_text_list.append(stemmed_text)

  return stemmed_text_list

In [291]:
X[0:5]

['@bbclaurak i agree Laura but the Party you seem to support so strongly is slowly doing the same thing . . . and usi… https://t.co/tsRsVBozIR',
 '@masturbacaolove Why?',
 '@JackRobinson80 @pgroresearch Yeah not great but quality is good!',
 'Penalty shit out Arsenal',
 'Obviously need some pen practice 🙈']

In [292]:
X_stemmed = stem_text(X)
X_stemmed[0:5]

['@bbclaurak i agre laura but the parti you seem to support so strongli is slowli do the same thing . . . and usi… https://t.co/tsrsvbozir',
 '@masturbacaolov why?',
 '@jackrobinson80 @pgroresearch yeah not great but qualiti is good!',
 'penalti shit out arsen',
 'obvious need some pen practic 🙈']

## Split Training Set and Testing Set

In [319]:
X_train, X_test, y_train, y_test = train_test_split(X_stemmed, y, train_size = 0.8, random_state = 42)

## Vectorize the text

In [320]:
counter = CountVectorizer()

counter.fit(X_stemmed)

train_counts = counter.transform(X_train)
test_counts = counter.transform(X_test)

In [321]:
list(counter.vocabulary_.items())[:20]

[('bbclaurak', 4204),
 ('agre', 2335),
 ('laura', 17670),
 ('but', 5617),
 ('the', 29673),
 ('parti', 22844),
 ('you', 33742),
 ('seem', 27017),
 ('to', 30195),
 ('support', 28891),
 ('so', 27890),
 ('strongli', 28666),
 ('is', 15484),
 ('slowli', 27763),
 ('do', 9226),
 ('same', 26614),
 ('thing', 29910),
 ('and', 2874),
 ('usi', 31410),
 ('https', 14509)]

# Build Model

In [322]:
model = MultinomialNB()

model.fit(train_counts, y_train)

MultinomialNB()

# Evaluate Model

In [323]:
train_pred = model.predict(train_counts)
print(f"Training set accuracy: {accuracy_score(y_train, train_pred):.2f}")

Training set accuracy: 0.93


In [324]:
test_pred = model.predict(test_counts)
print(f"Testing set accuracy: {accuracy_score(y_test, test_pred):.2f}")

Testing set accuracy: 0.69


In [325]:
print(classification_report(y_test, test_pred))

              precision    recall  f1-score   support

           0       0.67      0.69      0.68      1089
           1       0.68      0.64      0.66       949
           2       0.74      0.78      0.76       477

    accuracy                           0.69      2515
   macro avg       0.70      0.70      0.70      2515
weighted avg       0.69      0.69      0.69      2515



In [326]:
print(confusion_matrix(y_test, test_pred))

[[755 245  89]
 [303 607  39]
 [ 66  38 373]]
